# 🛠️ AWS Glue Migration Guide

This script **exports and imports** AWS Glue **databases, crawlers, and tables** between AWS regions.

## 📌 Features
✔ **Exports and imports** AWS Glue **databases, crawlers, and tables**  
✔ **Removes invalid parameters** (e.g., `CreatedBy`, `IsRegisteredWithLakeFormation`)  
✔ **Works across AWS regions** 🌍  
✔ **No AWS CLI required** (uses direct credentials in Boto3)  

In [12]:
import boto3
import json

# AWS Credentials (Replace with your actual credentials)
AWS_ACCESS_KEY = "ACCESS_KEY"
AWS_SECRET_KEY = "SECRET_KEY"

# AWS Account ID (Optional, for verification/logging)
AWS_ACCOUNT_ID = "AWS_ACCOUNT"

# Source and Target Regions
SOURCE_REGION = "ap-southeast-1"
TARGET_REGION = "ap-southeast-3"

# AWS Glue Clients for Source and Target Regions
glue_source = boto3.client(
    "glue",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=SOURCE_REGION
)

glue_target = boto3.client(
    "glue",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=TARGET_REGION
)

In [13]:
def migrate_database(database_name):
    """Export and import Glue database"""
    response = glue_source.get_database(Name=database_name)
    db_input = response["Database"]

    # Hilangin Attribut ga kepake
    db_input.pop("CreateTime", None)
    db_input.pop("CatalogId", None)

    # Import ke target region
    glue_target.create_database(DatabaseInput=db_input)
    print(f"✅ Database '{database_name}' migrated to {TARGET_REGION}.")

In [14]:
def migrate_crawler(crawler_name):
    """Export and import Glue crawler"""
    response = glue_source.get_crawler(Name=crawler_name)
    crawler_input = response["Crawler"]

    # Hilangin read only attribut
    keys_to_remove = [
        "State", "CreationTime", "LastUpdated", "CrawlElapsedTime",
        "LastCrawl", "Version", "Configuration", "SchemaChangePolicy",
        "RecrawlPolicy", "LineageConfiguration"
    ]
    
    for key in keys_to_remove:
        crawler_input.pop(key, None)

    # Import ke target region
    glue_target.create_crawler(**crawler_input)
    print(f"✅ Crawler '{crawler_name}' migrated to {TARGET_REGION}.")

In [15]:
def migrate_tables(database_name):
    """Export Import Glue Tabel"""
    response = glue_source.get_tables(DatabaseName=database_name)
    tables = response["TableList"]

    for table in tables:
        # Hilangin unused attribut
        keys_to_remove = [
            "CreateTime", "UpdateTime", "CatalogId", "VersionId",
            "DatabaseName", "CreatedBy", "IsRegisteredWithLakeFormation",
            "IsMultiDialectView"
        ]
        
        for key in keys_to_remove:
            table.pop(key, None)

        # Import ke target region
        glue_target.create_table(DatabaseName=database_name, TableInput=table)
        print(f"✅ Table '{table['Name']}' migrated to {TARGET_REGION}.")

In [16]:
DATABASE_NAME = "air-polution"
CRAWLER_NAME = "air-polution"

In [17]:
# Panggil FUngsi-nya
migrate_database(DATABASE_NAME)
migrate_crawler(CRAWLER_NAME)
migrate_tables(DATABASE_NAME)

print("\n🎉 AWS Glue migration completed successfully!")

✅ Database 'air-polution' migrated to ap-southeast-3.
✅ Crawler 'air-polution' migrated to ap-southeast-3.
✅ Table 'air_filter_data_csv' migrated to ap-southeast-3.

🎉 AWS Glue migration completed successfully!


## 🎯 4️⃣ Why This Works

✔ **Handles all Glue components** → Database, Crawler, and Tables  
✔ **Removes invalid parameters** → Avoids `ParamValidationError`  
✔ **Works across AWS regions** → Migrate **Glue** data efficiently 🌎  
✔ **Fully automated** → One script to **export and import everything** ⚡  

---

## 🎉 5️⃣ Final Notes

🚀 **Now, run the script and migrate your AWS Glue resources!**  
💡 **If you encounter any issues**, check the **AWS Glue Console** or run **AWS CLI commands** to validate.  